# Jet Substructure

## Load libraries and input files

In this notebook, we will load different files from different process. This can take some time:

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import hist
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import correctionlib
import copy

plt.style.use(hep.style.CMS)

In [ ]:
fname_qcd = "/eos/user/c/cmsdas/2024/short-ex-jet/RunIISummer20UL18NanoAODv9/QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8/NANOAODSIM/20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1/30000/792B4CD3-A001-F94F-9AAB-D74D532DE610.root"
fname_ttbar = "/eos/user/c/cmsdas/2024/short-ex-jet/RunIISummer20UL18NanoAODv9/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/2520000/130AF209-596A-BA46-BA9E-D8847511BB0F.root"
fname_rsgluon="/eos/user/c/cmsdas/2024/short-ex-jet/RunIISummer20UL18NanoAODv9/RSGluonToTT_M-1000_TuneCP5_13TeV-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/F2DC937C-90D2-CF40-9FCA-017BB8CD7B9E.root"
fname_wjets="/eos/user/c/cmsdas/2024/short-ex-jet/RunIISummer20UL18NanoAODv9/WJetsToQQ_HT-800toInf_TuneCP5_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v2/2550000/1651E9BC-5EFF-E149-A1C1-019EAA144F5E.root"

events_qcd = NanoEventsFactory.from_root( fname_qcd, schemaclass=NanoAODSchema.v6).events()
events_ttbar = NanoEventsFactory.from_root( fname_ttbar, schemaclass=NanoAODSchema.v6).events()
events_rsgluon = NanoEventsFactory.from_root( fname_rsgluon, schemaclass=NanoAODSchema.v6).events()
events_wjets = NanoEventsFactory.from_root( fname_wjets, schemaclass=NanoAODSchema.v6).events()

## Exercise 3.1: Jet Mass

Let's look a the different jet mass distributions for different processes:

In [ ]:
fig, ax = plt.subplots()

hists_mass = (
    hist.Hist.new
    .StrCat(["QCD", "ttbar", "RSGluon", "Wjets"], name="dataset")
    .Reg(50, 50, 300, name="mass", label='Fatjet softdrop mass')
    .Weight()
    .fill(
        dataset='QCD',
        mass=ak.flatten( events_qcd.FatJet.msoftdrop ),
    )
    .fill(
        dataset='ttbar',
        mass=ak.flatten( events_ttbar.FatJet.msoftdrop ),
    )
    .fill(
        dataset='RSGluon',
        mass=ak.flatten( events_rsgluon.FatJet.msoftdrop ),
    )
    .fill(
        dataset='Wjets',
        mass=ak.flatten( events_wjets.FatJet.msoftdrop ),
    )
).plot1d(density=True)

ax.legend()
# ax.set_yscale("log")

## Exercise 3.2: Jet Grooming algorithms

Let's make a plot comparing the ungroomed mass and the softdrop mass:

In [ ]:
hists_mass = (
    hist.Hist.new
    .StrCat(["mass", "msoftdrop"], name="mass_type")
    .Reg(200, 0, 200, name="mass", label='jet mass')
    .Weight()
    .fill(
        mass_type='mass',
        mass=ak.flatten(events_ttbar.FatJet.mass),
    )
    .fill(
        mass_type='msoftdrop',
        mass=ak.flatten(events_ttbar.FatJet.msoftdrop),
    )
)
fig, ax = plt.subplots()
hists_mass.plot1d()
ax.legend()

### $\rho$ parameter

Let's create this variable and make some comparison plots:

In [ ]:
fig, ax = plt.subplots()

hists_rho = (
    hist.Hist.new
    .StrCat(["QCD", "ttbar", "RSGluon", "Wjets"], name="dataset")
    .Reg(20, -10, 0, name="rho", label='Fatjet rho')
    .Weight()
    .fill(
        dataset='QCD',
        rho=ak.flatten( np.log( np.power( events_qcd.FatJet.msoftdrop / (events_qcd.FatJet.pt * 0.8) , 2 ) ) ),
    )
    .fill(
        dataset='ttbar',
        rho=ak.flatten(np.log( np.power( events_ttbar.FatJet.msoftdrop / (events_ttbar.FatJet.pt * 0.8) , 2 ) ) ),
    )
    .fill(
        dataset='RSGluon',
        rho=ak.flatten(np.log( np.power( events_rsgluon.FatJet.msoftdrop / (events_rsgluon.FatJet.pt * 0.8) , 2 ) ) ),
    )
    .fill(
        dataset='Wjets',
        rho=ak.flatten(np.log( np.power( events_wjets.FatJet.msoftdrop / (events_wjets.FatJet.pt * 0.8) , 2 ) ) ),
    )
).plot1d(density=True)

ax.legend()

## Exercise 3.3: Substructure Variables

Now, let's compare the different subtructure variables between different samples:

In [ ]:
def create_substructure_variables( jets ):
    
    jets['tau21'] = jets.tau2 / jets.tau1
    jets['tau32'] = jets.tau3 / jets.tau2

    return jets

In [ ]:
jets_qcd = create_substructure_variables( events_qcd.FatJet )
jets_ttbar = create_substructure_variables( events_ttbar.FatJet )
jets_rsgluon = create_substructure_variables( events_rsgluon.FatJet )
jets_wjets = create_substructure_variables( events_wjets.FatJet )

In [ ]:
hists = (
    hist.Hist.new
    .StrCat(["QCD", "ttbar", "RSGluon", "Wjets"], name="dataset")
    .Reg(200, 0, 2000, name="pt", label='Fatjet pt')
    .Reg(10, 0, 1, name="tau21", label='Fatjet tau21')
    .Reg(10, 0, 1, name="tau32", label='Fatjet tau32')
    .Reg(20, 0, 1, name="n2", label='Fatjet N2')
    .Reg(20, 0, 1, name="n3", label='Fatjet N3')
    .Weight()
    .fill(
        dataset='QCD',
        pt=ak.flatten(jets_qcd.pt),
        tau21=ak.flatten(jets_qcd.tau21),
        tau32=ak.flatten(jets_qcd.tau32),
        n2=ak.flatten(jets_qcd.n2b1),
        n3=ak.flatten(jets_qcd.n3b1),
    )
    .fill(
        dataset='ttbar',
        pt=ak.flatten(jets_ttbar.pt),
        tau21=ak.flatten(jets_ttbar.tau21),
        tau32=ak.flatten(jets_ttbar.tau32),
        n2=ak.flatten(jets_ttbar.n2b1),
        n3=ak.flatten(jets_ttbar.n3b1),
    )
    .fill(
        dataset='RSGluon',
        pt=ak.flatten(jets_rsgluon.pt),
        tau21=ak.flatten(jets_rsgluon.tau21),
        tau32=ak.flatten(jets_rsgluon.tau32),
        n2=ak.flatten(jets_rsgluon.n2b1),
        n3=ak.flatten(jets_rsgluon.n3b1),
    )
    .fill(
        dataset='Wjets',
        pt=ak.flatten(jets_wjets.pt),
        tau21=ak.flatten(jets_wjets.tau21),
        tau32=ak.flatten(jets_wjets.tau32),
        n2=ak.flatten(jets_wjets.n2b1),
        n3=ak.flatten(jets_wjets.n3b1),
    )
)

In [ ]:
fig, ax = plt.subplots()
hists.project("dataset", "tau21").plot1d(density=True)
ax.legend()

Let's do the same for tau32, but let's increase the jet pt to 450 GeV to enchance the efect:

In [ ]:
fig, ax = plt.subplots()
hists.project("dataset", "tau32", "pt")[:,:,450j::sum].plot1d(density=True)
ax.legend()

In [ ]:
fig, ax = plt.subplots()
hists.project("dataset", "n2", "pt")[:,:,300j::sum].plot1d(density=True)
ax.legend()

In [ ]:
fig, ax = plt.subplots()
hists.project("dataset", "n3", "pt")[:,:,450j::sum].plot1d(density=True)
ax.legend()

## Exercise 3.4: Taggers

### W tagging

Let's compare the tau2 / tau1 ratio for the AK8 jets from Standard Model top quarks to those from the QCD sample:

In [ ]:
hists_wtag = (
    hist.Hist.new
    .StrCat(["QCD", "ttbar"], name="dataset")
    .Reg(60, 0, 300, name="mass", label='Fatjet mass softdrop')
    .Reg(10, 0, 1, name="tau21", label='Fatjet tau21')
    .Reg(20, 0, 1, name="n2", label='Fatjet N2')
    .Reg(20, 0, 1, name="deepAK8_W", label='Fatjet deepAK8Jet W')
    .Weight()
    .fill(
        dataset='QCD',
        mass=ak.flatten(jets_qcd.msoftdrop),
        tau21=ak.flatten(jets_qcd.tau21),
        n2=ak.flatten(jets_qcd.n2b1),
        deepAK8_W=ak.flatten(jets_qcd.deepTagMD_WvsQCD),
    )
    .fill(
        dataset='ttbar',
        mass=ak.flatten(jets_ttbar.msoftdrop),
        tau21=ak.flatten(jets_ttbar.tau21),
        n2=ak.flatten(jets_ttbar.n2b1),
        deepAK8_W=ak.flatten(jets_ttbar.deepTagMD_WvsQCD),
    )
)

In [ ]:
fig, ax = plt.subplots()
hists_wtag.project("dataset", "tau21").plot1d(density=True)
ax.legend()

In [ ]:
fig, ax = plt.subplots()
hists_wtag.project("dataset", "mass").plot1d(density=True)
ax.legend()

What happen if we apply a selection in tau21 of $<0.5$?

In [ ]:
fig, ax = plt.subplots()
hists_wtag.project("dataset", "mass", "tau21")[:,:,:0.5j:sum].plot1d(density=True)
ax.legend()

Let's use now $N2$ with a selection of 0.2:

In [ ]:
fig, ax = plt.subplots()
hists_wtag.project("dataset", "mass", "n2")[:,:,0.2j::sum].plot1d(density=True)
ax.legend()

CMS has developed taggers using ML techinques. Let's try deepAK8 Jet tagger for W jets:

In [ ]:
fig, ax = plt.subplots()
hists_wtag.project("dataset", "deepAK8_W").plot1d(density=True)
ax.legend()

In [ ]:
fig, ax = plt.subplots()
hists_wtag.project("dataset", "mass", "deepAK8_W")[:,:,0.85j::sum].plot1d(density=True)
ax.legend()

### top tagging

Let's repeat the same exercise but now using the subtructure variables for top tagging:

In [ ]:
hists_toptag = (
    hist.Hist.new
    .StrCat(["QCD", "ttbar"], name="dataset")
    .Reg(60, 0, 300, name="mass", label='Fatjet mass softdrop')
    .Reg(10, 0, 1, name="tau32", label='Fatjet tau32')
    .Reg(20, 0, 1, name="n3", label='Fatjet N3')
    .Reg(20, 0, 1, name="deepAK8_top", label='Fatjet deepAK8Jet top')
    .Weight()
    .fill(
        dataset='QCD',
        mass=ak.flatten(jets_qcd.msoftdrop),
        tau32=ak.flatten(jets_qcd.tau32),
        n3=ak.flatten(jets_qcd.n3b1),
        deepAK8_top=ak.flatten(jets_qcd.deepTagMD_TvsQCD),
    )
    .fill(
        dataset='ttbar',
        mass=ak.flatten(jets_ttbar.msoftdrop),
        tau32=ak.flatten(jets_ttbar.tau32),
        n3=ak.flatten(jets_ttbar.n3b1),
        deepAK8_top=ak.flatten(jets_ttbar.deepTagMD_TvsQCD),
    )
)

In [ ]:
fig, ax = plt.subplots()
hists_toptag.project("dataset", "mass", "tau32")[:,:,:0.5j:sum].plot1d(density=True)
ax.legend()

In [ ]:
fig, ax = plt.subplots()
hists_toptag.project("dataset", "mass", "deepAK8_top")[:,:,0.85j::sum].plot1d(density=True)
ax.legend()

## Your turn

Now you can play with the subtructure variables and try to improve the jet mass distribution.